#### Workflow:
1. Create observations with 'create_data_points'. An observation is a data point in which
all signals 'peep', 'fio2', 'po2' are measured. All data points are rounded down to the
nearest hour. Hence each data points is an hour during which all signals where measured.
Notebook works with a single patient and hence '_create_data_points_batch' to make
testing faster. Consider changing the function name to 'create_observations'.

2. (to be implemented) Function 'select_cohort' selects the cohort by checking the
inclusion and exclusion criteria.

3. Create a table with the treatment. Function 'get_proning' creates a table with
proning sessions. This table will be used to check whether an observation was treated.


function that for each point checks preconditions

patients -> treated -> proning sessions -> included

patients -> never treated -> then look at points with blood gas -> if inclusion yes then data point

(update)

load blood gas points (create_data_points)
 -> drop ids that are not eligible (check_inclusion(drop = True))
 -> split points into (control - not proned and not proned after
                       measurement_control - not proned and wasn't proned in the past, may be proned in the future
                       trated - not proned will be proned after
                       measurement_treated - proned at the moment)


In [ ]:
%reset

In [ ]:
import os, sys, random

import pandas as pd
import numpy as np
import swifter

import pacmagic
import pacmagic_icu

from importlib import reload
from data_warehouse_utils.dataloader import DataLoader

os.chdir('/home/adam/files/causal_inference')
os.getcwd()

from causal_inference.experiment_generator.create_observations import create_data_points
from causal_inference.experiment_generator.create_observations import _get_hash_patient_id
from causal_inference.experiment_generator.create_observations import hour_rounder
from causal_inference.experiment_generator.create_observations import _create_data_points_batch
from causal_inference.experiment_generator.create_treatment import get_proning_table
from causal_inference.experiment_generator.create_treatment import proning_table_to_intervals


In [ ]:
# Reloads packages

reload(sys.modules['causal_inference'])
reload(sys.modules['causal_inference.experiment_generator'])
reload(sys.modules['causal_inference.experiment_generator.create_observations'])
reload(sys.modules['causal_inference.experiment_generator.create_treatment'])

from causal_inference.experiment_generator.create_observations import create_data_points
from causal_inference.experiment_generator.create_observations import _get_hash_patient_id
from causal_inference.experiment_generator.create_observations import hour_rounder
from causal_inference.experiment_generator.create_observations import _create_data_points_batch
from causal_inference.experiment_generator.create_treatment import get_proning_table
from causal_inference.experiment_generator.create_treatment import proning_table_to_intervals

In [ ]:
dl = DataLoader()

patient_id_all = _get_hash_patient_id(dl)

In [ ]:
ID = random.choice(patient_id_all)
ID

In [ ]:
df = _create_data_points_batch(dl = dl,
                               patient_id = ID,
                               compress = False, # doesn't have an effect yet
                               nearest = False) # We need to round down to be consistent

In [ ]:
df.head()

In [ ]:
# Remark: for some Ids there is no data, we need to account for it and then delete such a patient

df_treatment = get_proning_table(dl, ID, 16)

df_treatment.info()

In [ ]:
df_treatment.head()

In [ ]:
df.info()

In [ ]:
df['pf_ratio'].astype('int')

In [ ]:
df.head()

In [ ]:
df.columns